In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

C:\Users\yuval\AppData\Local\Temp\ipykernel_9444\1229163428.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [4]:
train_data = pd.read_csv("train.csv")

# The generated features:
# FamilySize - the size of the passenger's family
# Important-based on the name, whether or not the passenger has an important title
# RealCabin-the actual cabin letter
train_data['FamilySize'] = train_data['SibSp'] + train_data['Parch']
train_data['Important'] = train_data['Name'].str.contains('(Master.)|(Major.)|(Rev.)|(Dr.)|(Col.)|(Capt.)|(Countess.)|(Jonkheer.)')
train_data['RealCabin'] = train_data['Cabin'].str[0]
train_data['RealCabin'].fillna('X', inplace=True)
train_data['Age'].fillna(30.0, inplace=True)
train_data['Fare'].fillna(32.2, inplace=True)
features = ['Pclass', 'Important', 'Sex', 'Age', 'FamilySize', 'Fare', 'RealCabin', 'Embarked']

C:\Users\yuval\AppData\Local\Temp\ipykernel_9444\3932104050.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  train_data['Important'] = train_data['Name'].str.contains('(Master.)|(Major.)|(Rev.)|(Dr.)|(Col.)|(Capt.)|(Countess.)|(Jonkheer.)')
C:\Users\yuval\AppData\Local\Temp\ipykernel_9444\3932104050.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data['RealCabin'].fillna('X', inplace=True)
C:\Users\yuval\AppData\Local\Temp\ip

In [5]:
test_data = pd.read_csv("test.csv")
test_data['FamilySize'] = test_data['SibSp'] + test_data['Parch']
test_data['Important'] = test_data['Name'].str.contains('(Master.)|(Major.)|(Rev.)|(Dr.)|(Col.)|(Capt.)|(Countess.)|(Jonkheer.)')
test_data['RealCabin'] = test_data['Cabin'].str[0]
test_data['RealCabin'].fillna('X', inplace=True)
test_data['Age'].fillna(30.0, inplace=True)
test_data['Fare'].fillna(32.2, inplace=True)
features = ['Pclass', 'Important', 'Sex', 'Age', 'FamilySize', 'Fare', 'RealCabin', 'Embarked']

C:\Users\yuval\AppData\Local\Temp\ipykernel_9444\2344297327.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  test_data['Important'] = test_data['Name'].str.contains('(Master.)|(Major.)|(Rev.)|(Dr.)|(Col.)|(Capt.)|(Countess.)|(Jonkheer.)')
C:\Users\yuval\AppData\Local\Temp\ipykernel_9444\2344297327.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_data['RealCabin'].fillna('X', inplace=True)
C:\Users\yuval\AppData\Local\Temp\ipyker

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

y = train_data["Survived"]

features = ["Pclass", "Important", "Sex", "Age", "FamilySize", "Fare", "Embarked"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

X_learn, X_valid, y_learn, y_valid = train_test_split(X, y, test_size = 0.2, random_state = 1)
for j in range(1,11):
    model = RandomForestClassifier(n_estimators=100, max_depth=j, random_state=1)
    model.fit(X_learn, y_learn)
    predictions = model.predict(X_valid)
    y_valid_numpy = y_valid.to_numpy()
    sum = 0
    for i in range(len(predictions)):
        if predictions[i] == y_valid_numpy[i]:
            sum += 1
    print(j, " Random forest ", sum/len(predictions))

for j in range(1,10):
    model = KNeighborsClassifier(n_neighbors=j)
    model.fit(X_learn, y_learn)
    predictions = model.predict(X_valid)
    y_valid_numpy = y_valid.to_numpy()
    sum = 0
    for i in range(len(predictions)):
        if predictions[i] == y_valid_numpy[i]:
            sum += 1
    print(j, " NN ", sum/len(predictions))

model = svm.SVC(kernel='linear')
model.fit(X_learn, y_learn)
predictions = model.predict(X_valid)
y_valid_numpy = y_valid.to_numpy()
sum = 0
for i in range(len(predictions)):
    if predictions[i] == y_valid_numpy[i]:
        sum += 1
print("SVM ", sum/len(predictions))

model = RandomForestClassifier(n_estimators=100, max_depth=6, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

1  Random forest  0.776536312849162
2  Random forest  0.7821229050279329
3  Random forest  0.7988826815642458
4  Random forest  0.7988826815642458
5  Random forest  0.770949720670391
6  Random forest  0.770949720670391
7  Random forest  0.7821229050279329
8  Random forest  0.7821229050279329
9  Random forest  0.7877094972067039
10  Random forest  0.8100558659217877
1  NN  0.6927374301675978
2  NN  0.6871508379888268
3  NN  0.7541899441340782
4  NN  0.6871508379888268
5  NN  0.7541899441340782
6  NN  0.6927374301675978
7  NN  0.7206703910614525
8  NN  0.7039106145251397
9  NN  0.7039106145251397
SVM  0.7877094972067039
Your submission was successfully saved!
